# Clustering polls into topics
> Using titles/descriptions of polls for clustering. Goal is unsupervised grouping into topicsish based on [this](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py).

> Note: You may need to run `python -m spacy download de_core_news_sm`, if not already done, to process the German language.

TODO: implement tfidf version of the below

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bundestag import poll_clustering as pc
from bundestag import abgeordnetenwatch as aw
import pprint

In [ ]:
legislature_id = 111

In [ ]:
%%time
df_polls = aw.get_polls_df(legislature_id)
df_polls.head()

## Clustering based on poll title



Sanity checking word counts, longest and shortest titles

### Cleaning using spacy

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

In [ ]:
%%time
col = 'poll_title'
nlp_col = f'{col}_nlp_processed'
st = pc.SpacyTransformer()
df_polls[nlp_col] = df_polls.pipe(st.clean_text, col=col)
pc.test_cleaned_text(df_polls, col=nlp_col)

In [ ]:
df_polls.head()

### Inspecting word frequencies

In [ ]:
pc.compare_word_frequencies(df_polls, col, nlp_col)

The word count distribution shifted to lower values, as could be expected, but no documents were left without any words.

### Transforming using LDA

In [ ]:
%%time
st.fit(df_polls[nlp_col].values, mode='lda', num_topics=10)
print('Discovered topics:')
pprint.pprint(st.lda_topics)

In [ ]:
df_lda = st.transform_documents(df_polls[nlp_col])
df_lda.head()

In [ ]:
pc.test_dense_topic_scores(df_lda.values)

In [ ]:
%%time
df_polls, nlp_feature_cols = df_polls.pipe(st.transform, col=nlp_col, return_new_cols=True)
df_polls.head()

In [ ]:
%%time
pc.pca_plot_lda_topics(df_polls, st, col, nlp_feature_cols)

### Transforming using TFIDF

In [ ]:
%%time
# tfidf_model = gensim.models.TfidfModel(corpus=corpus)